# Zero Shot Topic Classification on CORD-19

## Introduction

In this notebook we'll build a Zero Shot Topic Classifier on the COVID-19 Open Research Dataset (CORD-19, Wang et al., 2020).
Essentially, we aim to build a web application capable of receiving natural language questions, such as "what do we know about vaccines and therapeutics?", and then displaying the most relevant research literature regarding the specific question.
This dataset has received wide attention in the data mining and natural language processing community in order to develop tools to aid health workers stay up-to-date with the latest and most relevant research about the current pandemic.

Recent advances in NLP, such as OpenAI's GPT-3 (Brown et al., 2020), have shown that large language models can achieve competitive performance on downstream tasks with less task-specific data than it'd be required by smaller models.
However, GPT-3 is currently difficult to use on real world applications due to its size of ~175 billions of parameters.

Recent experiments made at HuggingFace (Davison, 2020) explored the potential of using Sentence-BERT (Reimers and Gurevych, 2020) to separately embed sentences and never-seen-before topic labels.
Then, they'd rank the sentence's topics by measuring the cosine distance between both vectors (Veeranna, 2016), obtaining promising results.

In another experiment, they use a pre-trained natural languange inference (NLI) sequence-pair classifier as an out of-the-box zero shot text classifier, as proposed by Yin et al. (2020).
By using a pre-trained BART model (Lewis et al., 2019) fine-tuned on the Multigenre NLI corpus, they were able to score an F1 score of 53.7 on the Yahoo News dataset.
The dataset has 10 classes and the current supervised models state of the art is an accuracy of 77.62.

## Cosine similarity method

First, we'll use Sentence-BERT to embed both the papers and the never-seen-before question in order to measure the cosine similarity and assess the papers relevance to the question.
For the sake of efficiency, we'll iterate over the dataset and precompute the papers representations using their title and abstract.
Then, we'll compute the cosine distance between the embedded question and each one of the papers representations.
Finally, the papers might be ordered by their distances in order to first display those that are relevant to the question.

This method has been implemented at https://risotto.inria.cl/zero-shot.

In [ ]:
# %env CUDA_VISIBLE_DEVICES=4,5,6

env: CUDA_VISIBLE_DEVICES=4,5,6


In [ ]:
# default_exp zero_shot

In [ ]:
# all_flag

In [ ]:
# export

import math

import pandas as pd
import numpy as np
import scipy
from sentence_transformers import SentenceTransformer
from fastprogress import progress_bar


def embed_papers(papers):
    # papers = load_papers_artifact()
    model = SentenceTransformer("bert-base-nli-mean-tokens")

    batch_size = 256
    num_rows = len(papers)
    num_batches = math.ceil(num_rows / batch_size)

    embeddings = pd.Series([], dtype=object, name="embeddings")

    for batch_id in progress_bar(range(num_batches)):
        # Concatenate title and abstract
        start_idx = batch_id * batch_size
        end_idx = start_idx + batch_size
        slice_df = papers.iloc[start_idx:end_idx]
        title_abstract = (slice_df.title + ". " +
                          slice_df.abstract).fillna("").values.tolist()

        # Embed
        sentence_embeddings = model.encode(title_abstract)

        for i, (paper_idx, _) in enumerate(slice_df.iterrows()):
            embeddings.at[paper_idx] = sentence_embeddings[i]

    return embeddings


def cosine_distance_query(model, query, papers_embeddings):
    query_encoded = model.encode([query])
    papers_array = np.array(papers_embeddings.to_list())
    distances = scipy.spatial.distance.cdist(query_encoded, papers_array,
                                             "cosine")[0]
    distances_series = pd.Series(distances,
                                 index=papers_embeddings.index,
                                 name="distances")

    return distances_series


def get_sentence_transformer(name="bert-base-nli-mean-tokens"):
    return SentenceTransformer(name)

## Natural Language Inference (NLI) method

In this approach we use a BART classifier (Lewis et al., 2019) pre-trained on the Multi-Genre NLI (MultiNLI, Williams et al., 2018) corpus as the base model.

Given research interests expressed in natural language, we pose the problem of recovering relevant research from the CORD-19 dataset (Wang et al., 2020) as a Zero Shot Topic Classification task (Yin et al., 2019).
Leveraging the Natural Language Inference task framework, we assess each paper relevance by feeding the model with the paper's title and abstract as premise and a research interest as hypothesis.

Finally, we use the model's entailment inference values as proxy relevance scores for each paper.

In [ ]:
# export

import math
from transformers import BartForSequenceClassification, BartTokenizer


def get_nli_model(name="facebook/bart-large-mnli"):
    model = BartForSequenceClassification.from_pretrained(name)
    tokenizer = BartTokenizer.from_pretrained(name)
    return model, tokenizer

/home/lmarti/risotto/venv-risotto/lib/python3.7/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
from risotto.artifacts import load_papers_artifact

try:
    papers = load_papers_artifact()
    model, tokenizer = get_nli_model()
except:
    print('Data is not ready.')

/Users/lmarti/.pyenv/versions/3.8.2/envs/risotto/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/Users/lmarti/.pyenv/versions/3.8.2/envs/risotto/lib/python3.8/site-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Data is not ready.


In [ ]:
# export

import numpy as np
from fastprogress.fastprogress import progress_bar


def build_tokenized_papers_artifact(papers,
                                    tokenizer,
                                    should_dump=True,
                                    dump_path=None,
                                    batch_size=128):
    num_batches = math.ceil(len(papers) / batch_size)
    tokenized_series = pd.Series([], dtype=object, name="tokenized_papers")

    for batch_idx in progress_bar(range(num_batches)):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        papers_batch = papers.iloc[start_idx:end_idx]

        title_abstract = (papers_batch.title + ". " +
                          papers_batch.abstract).fillna(" ").values.tolist()
        tokenized_batch = tokenizer.batch_encode_plus(
            title_abstract, max_length=tokenizer.model_max_length)

        for i, (paper_idx, _) in enumerate(papers_batch.iterrows()):
            tokenized_series.at[paper_idx] = tokenized_batch["input_ids"][i]

    if should_dump:
        tokenized_series.to_hdf(dump_path, key="tokenized_papers")

    return tokenized_series


def load_tokenized_papers_artifact(artifacts_path):
    return pd.read_hdf(artifacts_path, key="tokenized_papers")

In [ ]:
# Build tokenized papers
tokenized_papers = build_tokenized_papers_artifact(
    papers=papers,
    tokenizer=tokenizer,
    dump_path="artifacts/nli_artifacts.hdf")
tokenized_papers.head()

ug7v899j    [0, 20868, 1575, 9, 2040, 12, 32012, 1308, 438...
02tnwd4m    [0, 19272, 4063, 30629, 35, 10, 1759, 12, 3382...
ejv2xln0    [0, 6544, 24905, 927, 8276, 12, 495, 8, 34049,...
2b73a28n    [0, 21888, 9, 253, 15244, 2614, 12, 134, 11, 1...
9785vg6d    [0, 13120, 8151, 11, 22201, 44828, 4590, 11, 1...
Name: tokenized_papers, dtype: object

In [ ]:
# Load tokenized papers
tokenized_papers = load_tokenized_papers_artifact("artifacts/nli_artifacts.hdf")
tokenized_papers.head()

ug7v899j    [0, 20868, 1575, 9, 2040, 12, 32012, 1308, 438...
02tnwd4m    [0, 19272, 4063, 30629, 35, 10, 1759, 12, 3382...
ejv2xln0    [0, 6544, 24905, 927, 8276, 12, 495, 8, 34049,...
2b73a28n    [0, 21888, 9, 253, 15244, 2614, 12, 134, 11, 1...
9785vg6d    [0, 13120, 8151, 11, 22201, 44828, 4590, 11, 1...
Name: tokenized_papers, dtype: object

In [ ]:
# export

import torch


def build_entailments_artifact(tokenized_papers,
                               query_tokenized,
                               batch_size=64,
                               device="cuda",
                               should_dump=True,
                               dump_path=None):
    # query_encoded = [2, 2, *tokenizer.encode(query)[1:]]
    query_encoded = [2, 2, *query_tokenized[1:]]

    model.eval()
    model.to(device)

    num_batches = math.ceil(len(tokenized_papers) / batch_size)
    entail_probs = []

    for batch_idx in progress_bar(range(num_batches)):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        tokenized_papers_batch = tokenized_papers.iloc[
            start_idx:end_idx].tolist()
        max_length = float("-inf")
        for i in range(len(tokenized_papers_batch)):
            tokenized_paper = tokenized_papers_batch[i]
            # Ugly hack...
            tokenized_papers_batch[i] = [
                *tokenized_paper[:-1][:(tokenizer.model_max_length -
                                        len(query_encoded))], *query_encoded
            ]
            tokenized_paper = tokenized_papers_batch[i]
            if len(tokenized_paper) > max_length:
                max_length = len(tokenized_paper)
        masks = []
        for tokenized_paper in tokenized_papers_batch:
            paper_length = len(tokenized_paper)
            delta = max_length - paper_length
            tokenized_paper += [1 for _ in range(delta)]
            mask = [1 for _ in range(paper_length)]
            mask += [0 for _ in range(delta)]
            masks.append(mask)

        input_ids = torch.tensor(tokenized_papers_batch).to(device)
        attention_mask = torch.tensor(masks).to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)[0]

        entail_contradiction_logits = outputs[:, [0, 2]]
        probs = entail_contradiction_logits.softmax(dim=1)
        entail_probs += (probs[:, 1] * 100).tolist()

    entail_series = pd.Series(entail_probs,
                              index=tokenized_papers.index,
                              name="entailments")

    if should_dump:
        entail_series.to_hdf(dump_path, key="entailments")

    return entail_series


def load_entailments_artifact(artifacts_path):
    return pd.read_hdf(artifacts_path, key="entailments")

## References

Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., Neelakantan, A., Shyam, P., Sastry, G., Askell, A., Agarwal, S., Herbert-Voss, A., Krueger, G., Henighan, T., Child, R., Ramesh, A., Ziegler, D. M., Wu, J., Winter, C., … Amodei, D. (2020). Language Models are Few-Shot Learners. https://arxiv.org/abs/2005.14165

Davison, J. (2020). Zero-Shot Learning in Modern NLP. https://joeddav.github.io/blog/2020/05/29/ZSL.html

Lewis, M., Liu, Y., Goyal, N., Ghazvininejad, M., Mohamed, A., Levy, O., Stoyanov, V., & Zettlemoyer, L. (2019). BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension. http://arxiv.org/abs/1910.13461

Reimers, N., & Gurevych, I. (2020). Sentence-BERT: Sentence embeddings using siamese BERT-networks. EMNLP-IJCNLP 2019 - 2019 Conference on Empirical Methods in Natural Language Processing and 9th International Joint Conference on Natural Language Processing, Proceedings of the Conference, 3982–3992. https://doi.org/10.18653/v1/d19-1410

Veeranna, S. P., Nam, J., Mencía, E. L., & Fürnkranz, J. (2016). Using semantic similarity for multi-label zero-shot classification of text documents. ESANN 2016 - 24th European Symposium on Artificial Neural Networks, April, 423–428.

Wang, L. L., Lo, K., Chandrasekhar, Y., Reas, R., Yang, J., Eide, D., Funk, K., Kinney, R., Liu, Z., Merrill, W., Mooney, P., Murdick, D., Rishi, D., Sheehan, J., Shen, Z., Stilson, B., Wade, A. D., Wang, K., Wilhelm, C., … Kohlmeier, S. (2020). CORD-19: The Covid-19 Open Research Dataset. https://arxiv.org/abs/2004.10706

Williams, A., Nangia, N., & Bowman, S. R. (2018). A Broad-Coverage Challenge Corpus for Sentence Understanding through Inference. Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Human Language Technologies, Volume 1 (Long Papers), 1112--1122. http://aclweb.org/anthology/N18-1101

Yin, W., Hay, J., & Roth, D. (2019). Benchmarking zero-shot text classification: Datasets, evaluation and entailment approach. EMNLP-IJCNLP 2019 - 2019 Conference on Empirical Methods in Natural Language Processing and 9th International Joint Conference on Natural Language Processing, Proceedings of the Conference, 3914–3923. https://doi.org/10.18653/v1/d19-1404

---